In [1]:
import os
import time
from typing import Tuple, Iterator

from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import tensorflow as tf
import tensorflow_datasets as tfds

import common

2024-07-24 01:17:08.686308: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-24 01:17:08.686374: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-24 01:17:08.687234: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-24 01:17:08.692609: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-24 01:17:09.723154: W tensorflow/compiler/tf2

Data preparation

In [2]:
# tf.debugging.set_log_device_placement(True)
# tf.config.list_physical_devices("GPU")

In [3]:
data_path = "../card_images"
image_pathes = common.extract_images(data_path)
labels = []
for index, image_path in enumerate(image_pathes):
    basename = os.path.basename(image_path)
    id, _ = os.path.splitext(basename)
    labels.append(id)

In [4]:
def load_image_from_path(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float16)
    return image, label

In [5]:
image_dataset = tf.data.Dataset.from_tensor_slices(image_pathes)
label_dataset = tf.data.Dataset.from_tensor_slices(labels)

dataset = tf.data.Dataset.zip((image_dataset, label_dataset))
dataset = dataset.map(load_image_from_path, num_parallel_calls=tf.data.AUTOTUNE)

2024-07-24 01:17:11.274603: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-24 01:17:11.321356: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-24 01:17:11.321401: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-24 01:17:11.324582: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-24 01:17:11.324634: I external/local_xla/xla/stream_executor

Prepare training

In [6]:
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Flatten
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

In [28]:
# IMAGE_SHAPE = (1185, 813, 3) # big szie
IMAGE_SHAPE = (268, 391, 3) # small size

In [29]:
backbone = EfficientNetV2B0(include_top=False,
                            weights="imagenet")

model = Sequential([
    InputLayer(input_shape=IMAGE_SHAPE),
    backbone,
    Flatten(),
    Dense(100, activation='softmax')
])

In [30]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-b0 (Functio  (None, None, None, 1280   5919312   
 nal)                        )                                   
                                                                 
 flatten_8 (Flatten)         (None, 149760)            0         
                                                                 
 dense_8 (Dense)             (None, 100)               14976100  
                                                                 
Total params: 20895412 (79.71 MB)
Trainable params: 20834804 (79.48 MB)
Non-trainable params: 60608 (236.75 KB)
_________________________________________________________________


In [11]:
loss_fn = SparseCategoricalCrossentropy(from_logits=True)